This notebook reproduces example from Cameron and Trivedi (2010) Microeconometrics Using Stata, pp.364-7

In [3]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import poisson

import os
#os.chdir(r"C:\Users\Randall\OneDrive\Documents\STATA\mus")

### Get the data

In [4]:
data = pd.read_stata('http://cameron.econ.ucdavis.edu/musbook/mus10data.dta')
data = data[data['year02']==1]
data['cons'] = 1
y = data['docvis'].values
X = data[['private', 'chronic', 'female', 'income', 'cons']].values

## Solve using scipy

Define the (negative) of the likelihood function

In [5]:
def loglikelihood(b):
    μ = np.exp(X @ b)
    return -poisson.logpmf(y, μ).sum()

Solve from an initial value 

In [6]:
?minimize

In [7]:
b = np.array([0.0, 0.0, 0.0, 0.0, 1.0])
res = minimize(loglikelihood, b)
print(res.x)

[ 0.79866596  1.09186514  0.49254788  0.00355699 -0.22972604]


Compute marginal effects

In [8]:
xmg0 = np.tile(X.mean(axis=0), [4,1])
xmg1 = np.tile(X.mean(axis=0), [4,1])
for i in range(3):
    xmg0[i,i] = 0
    xmg1[i,i] = 1
xmg1[3, 3] = 2 * xmg0[3, 3]

mg = pd.DataFrame(np.c_[np.exp(xmg0@ res.x), np.exp(xmg1@ res.x)].round(1),
                  index=['private', 'chronic', 'female', 'income'])

mg

,0,1
private,1.6,3.6
chronic,2.1,6.3
female,2.4,3.9
income,3.0,3.4


## Solve using Newton Raphson

In [9]:
Δ = 1.0
β = np.array([[0,0,0,0,1]]).T
y = data[['docvis']].values # extracted as dataframe (not series) so we get 2-dimensional column vector

In [10]:
iter = 0
while Δ > 1e-16:
    μ = np.exp(X @ β)
    grad = X.T @ (y-μ)
    H = X.T @ np.diag(μ.flat) @ X
    βold = β
    β = βold + np.linalg.solve(H, grad)
    Δ = np.square(βold-β).sum() / np.square(βold).sum()
    iter += 1

print(β.T)    

[[ 0.79866538  1.09186511  0.49254807  0.00355701 -0.22972634]]


## Solve using compecon

In [11]:
from compecon import OP

In [12]:
y = data['docvis'].values # extracted as series (not dataframe) so we get 1-dimensional vector

def loglikelihood(b):
    print(b.dtype)
    μ = np.exp(X @ b)
    return poisson.logpmf(y, μ).sum()

In [13]:
OP(loglikelihood)


A container class:

	      A = None
	    _x0 = [None]
	_x_list = []
	errcode = True
	      f = <function OP.__init__.<locals>.<lambda> at 0x0000020E1B07FCA0>
	      n = 1
	   opts = Options for solving a Unconstraint Optimization Problem:

	SearchMeth = bfgs
	  StepMeth = bt
	     all_x = False
	      eps0 = 1.0
	      eps1 = 1e-12
	     maxit = 250
	  maxsteps = 50
	      show = False
	       tol = 1.4901161193847656e-08

	  reset = False
	      x = None

In [14]:
β = np.array([0.0,0,0,0,1])
loglikelihood(β)
OP(loglikelihood,maxit =25).qnewton(β, show=True)

float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
   0     0  1.89e+08
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
flo

ValueError: NaNs or Infs encountered